In [1]:
pip install selenium bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import necessary modules

import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import pandas as pd

In [3]:
#Initiate browser to scrape, navigate to Linkedin main website

PATH = 'C:/Users/91836/data_science_proj/internA/chromedriver.exe'
browser = webdriver.Chrome(PATH)
browser.get('https://www.linkedin.com/')
browser.maximize_window()
time.sleep(5)

In [4]:
#Login procedure
username = browser.find_element('id', 'session_key')
username.send_keys('shruti_mistry@yahoo.com')
password = browser.find_element('id', 'session_password')
password.send_keys('Shrutii@9')
browser.find_element(By.CLASS_NAME,'sign-in-form__submit-button').click()
time.sleep(15)

In [5]:
#Navigate to job listing subdomain

browser.get('https://www.linkedin.com/jobs/search/')
time.sleep(5)

In [6]:
#Search for a specific job and location
job_search = browser.find_element(By.CLASS_NAME, "jobs-search-box__text-input")
job_search.send_keys('data scientist')
job_loc_search = browser.find_element(By.XPATH, '//input[@aria-label="City, state, or zip code"]')
job_loc_search.clear()
job_loc_search.send_keys('Maharashtra')
browser.find_element(By.CLASS_NAME, "jobs-search-box__submit-button").click()
time.sleep(20)

In [7]:
#Start scraping job details
#Initiate an empty list to hold job infomation

job_lists = []
for page in range(2,3): # range(2, can be change depending on mood) :)
    job_cards = browser.find_elements(By.XPATH, '//ul[@class="scaffold-layout__list-container"]/li') #Class got changed??
    card_ids = [result.get_attribute('id') for result in job_cards] #Get all job ids to let selenium click each job individually
    #print(card_ids)
    for id in card_ids:
        time.sleep(1)
        try:
            job = browser.find_element(By.ID, id)
            job_id = job.get_attribute('data-occludable-job-id')
            browser.find_element(By.XPATH, f'//div[@data-job-id={job_id}]').click() #Click on each job for the html to show the card
        except:
            continue

        try:
            job_title = browser.find_element(By.XPATH, "//h2[@class='t-24 t-bold jobs-unified-top-card__job-title']").text
        except:
            job_title = ''
           

        try:
            sub_card_1 = browser.find_element(By.XPATH, "//span[@class='jobs-unified-top-card__subtitle-primary-grouping t-black']").find_elements(By.TAG_NAME, 'span')
            job_company = sub_card_1[0].text
            job_location = sub_card_1[1].text
        except:
            job_company = ''
            job_location = ''
            

        job_list = [job_title, job_company, job_location]
        job_lists.append(job_list)

    browser.find_element(By.XPATH, f'//button[@aria-label="Page {page}"]').click()
    time.sleep(2)

In [8]:
#Create a dataframe for exporting csv file

df = pd.DataFrame(job_lists, columns=['Job title', 'Company', 'Working location'])
df

#Export dataframe to a csv file
df.to_csv('linkedin_job_details.csv')